# Import packages

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q datasets
!pip install accelerate -U
!pip install evaluate
!pip install transformers
! pip install snorkel
! pip install nltk
! pip install utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 1.7 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, TrainerCallback, AutoModelForCausalLM, AutoModelForSeq2SeqLM,BitsAndBytesConfig, DataCollatorWithPadding
from datasets import load_dataset, concatenate_datasets, load_from_disk, load_metric, Dataset, ClassLabel
import numpy as np
import pandas as pd

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


# Data Loading and Transformation

In [ ]:
# Load data
df_total = pd.read_parquet('/content/drive/MyDrive/Dissertation/Data/df.parquet')
# One-shot sample
df_label_unique_sample = df_total.groupby('label_cat', group_keys=False).apply(lambda df: df.sample(1))
# Create label mapping
label = list(df_label_unique_sample['label'])
id = list(df_label_unique_sample['label_cat'])
label_to_id = dict(zip(label,id))
id_to_label = dict(zip(id,label))
# Define mapping
Mapping = ClassLabel(names= label)

# Load dataset
# Load the parquet dataset
dataset = load_dataset("parquet", data_files={'train': '/content/drive/MyDrive/Dissertation/Data/train_df.parquet', 'test': '/content/drive/MyDrive/Dissertation/Data/test_df.parquet'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
# Filter out most labeled data
train_ds = dataset['train']
# Rename columns to the suitable column names for modelling
test_ds = dataset['test'].remove_columns('__index_level_0__')
train_ds = dataset['train'].remove_columns('__index_level_0__')
train_ds = train_ds.rename_column("processed_text", "text")
train_ds = train_ds.rename_column("label_cat", "labels")
test_ds = test_ds.rename_column("processed_text", "text")
test_ds= test_ds.rename_column("label_cat", "labels")
# Filter
train_ds_filtered = train_ds.filter(lambda example: example["labels"] != 11)
train_ds_filtered = train_ds_filtered.filter(lambda example: example["labels"] != 12)

Filter:   0%|          | 0/261 [00:00<?, ? examples/s]

Filter:   0%|          | 0/225 [00:00<?, ? examples/s]

In [ ]:
#Rename columns
df_total.rename(columns={'processed_text':'text'},inplace=True)
df_total.rename(columns={'label':'labels'},inplace=True)
df_total.drop('label_cat',axis = 1,inplace=True)

In [ ]:
# Show without truncation
import pandas as pd
DISPLAY_ALL_TEXT = True
pd.set_option("display.max_colwidth", 0 if DISPLAY_ALL_TEXT else 50)

# Write TF function (Adapted from Snorkel Official document)

In [ ]:
# REF: https://github.com/snorkel-team/snorkel-tutorials/blob/master/spam/02_spam_data_augmentation_tutorial.ipynb

In [ ]:
from snorkel.preprocess.nlp import SpacyPreprocessor
import nltk
from nltk.corpus import wordnet as wn
from collections import OrderedDict
from snorkel.augmentation import transformation_function
spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)
dler = nltk.downloader.Downloader()
dler._update_index()
dler.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# replace entity name

def get_synonym(word, pos=None):
    """Get synonym for word given its part-of-speech (pos)."""
    synsets = wn.synsets(word, pos=pos)
    # Return None if wordnet has no synsets (synonym sets) for this word and pos.
    if synsets:
        words = [lemma.name() for lemma in synsets[0].lemmas()]
        if words[0].lower() != word.lower():  # Skip if synonym is same as word.
            return words[0].replace("_", " ")

def replace_token(spacy_doc, idx, replacement):
    """Replace token in position idx with replacement."""
    return " ".join([spacy_doc[:idx].text, replacement, spacy_doc[1 + idx :].text])

@transformation_function(pre=[spacy])
def replace_noun_with_synonym(x):
    # Get indices of noun tokens in sentence.
    noun_idxs = [i for i, token in enumerate(x.doc) if token.pos_ == "NOUN"]
    if noun_idxs:
        # Pick random noun idx to replace.
        idx = np.random.choice(noun_idxs)
        synonym = get_synonym(x.doc[idx].text, pos="n")
        # If there's a valid noun synonym, replace it. Otherwise, return None.
        if synonym:
            x.text = replace_token(x.doc, idx, synonym)
            return x

@transformation_function(pre=[spacy])
def replace_verb_with_synonym(x):
    # Get indices of verb tokens in sentence.
    verb_idxs = [i for i, token in enumerate(x.doc) if token.pos_ == "VERB"]
    if verb_idxs:
        # Pick random verb idx to replace.
        idx = np.random.choice(verb_idxs)
        synonym = get_synonym(x.doc[idx].text, pos="v")
        # If there's a valid verb synonym, replace it. Otherwise, return None.
        if synonym:
            x.text = replace_token(x.doc, idx, synonym)
            return x
@transformation_function(pre=[spacy])
def replace_adjective_with_synonym(x):
    # Get indices of adjective tokens in sentence.
    adjective_idxs = [i for i, token in enumerate(x.doc) if token.pos_ == "ADJ"]
    if adjective_idxs:
        # Pick random adjective idx to replace.
        idx = np.random.choice(adjective_idxs)
        synonym = get_synonym(x.doc[idx].text, pos="a")
        # If there's a valid adjective synonym, replace it. Otherwise, return None.
        if synonym:
            x.text = replace_token(x.doc, idx, synonym)
            return x

In [ ]:
tfs = [
    replace_verb_with_synonym,
    replace_adjective_with_synonym,
    replace_noun_with_synonym
]

In [ ]:
def preview_tfs(df, tfs):
    transformed_examples = []
    for f in tfs:
        for i, row in df.sample(frac=1, random_state=2).iterrows():
            transformed_or_none = f(row)
            # If TF returned a transformed example, record it in dict and move to next TF.
            if transformed_or_none is not None:
                transformed_examples.append(
                    OrderedDict(
                        {
                            "TF Name": f.name,
                            "Original Text": row.text,
                            "Transformed Text": transformed_or_none.text,
                        }
                    )
                )
                break
    return pd.DataFrame(transformed_examples)

In [ ]:
preview_tfs(df_total, tfs)

,TF Name,Original Text,Transformed Text
0,replace_verb_with_synonym,redecorate interior demised premise good workmanlike manner using material good quality often may reasonable opinion landlord,redecorate interior demised premise good workmanlike manner use material good quality often may reasonable opinion landlord
1,replace_adjective_with_synonym,perpetuity period `` mean period eighty year date,perpetuity period `` average period eighty year date
2,replace_noun_with_synonym,yearly rent payable lease reviewed expiry fifth tenth year term,yearly rent payable lease reviewed termination fifth tenth year term


# Apply to dataset

In [ ]:
from snorkel.augmentation import MeanFieldPolicy
def mean_policy(N_sample):
  mean_field_policy = MeanFieldPolicy(
      len(tfs),
      sequence_length=2,
      n_per_original=N_sample,
      p= [0.2,0.4,0.4],
      keep_original=False)
  return mean_field_policy

In [ ]:
from transformers.models.t5.modeling_tf_t5 import TF_T5_PRETRAINED_MODEL_ARCHIVE_LIST
import datasets
train_ds_filtered = train_ds_filtered.to_pandas()

In [ ]:
from snorkel.augmentation import PandasTFApplier
def apply_to_dataset(N_sample):
  tf_applier = PandasTFApplier(tfs, mean_policy(N_sample))
  df_train_augmented = tf_applier.apply(train_ds_filtered)
  Y_train_augmented = df_train_augmented["labels"].values
  return df_train_augmented
  # Concatenate X and Y variable

In [ ]:
df_train_augmented_200 = apply_to_dataset(2)
df_train_augmented_300 = apply_to_dataset(3)
df_train_augmented_400 = apply_to_dataset(4)

100%|██████████| 211/211 [00:01<00:00, 156.03it/s]


In [ ]:
# Save to disk
def save_to_disk(dataframe):
  text = dataframe['text']
  label = dataframe['labels']
  synthetic_dataset = Dataset.from_dict({'text': text, 'labels': label})
  return synthetic_dataset

In [ ]:
output_path = '/content/drive/MyDrive/Dissertation/GPT_data/Snorkel_data'
save_to_disk(df_train_augmented_200).save_to_disk(output_path + '/' + '200')
save_to_disk(df_train_augmented_300).save_to_disk(output_path + '/' + '300')
save_to_disk(df_train_augmented_400).save_to_disk(output_path + '/' + '400')

Saving the dataset (0/1 shards):   0%|          | 0/118 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/183 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/230 [00:00<?, ? examples/s]

# Load training data

In [ ]:
snorkel_ds_200 = Dataset.from_file('/content/drive/MyDrive/Dissertation/GPT_data/Snorkel_data/200/data-00000-of-00001.arrow')
snorkel_ds_300 = Dataset.from_file('/content/drive/MyDrive/Dissertation/GPT_data/Snorkel_data/300/data-00000-of-00001.arrow')
snorkel_ds_400 = Dataset.from_file('/content/drive/MyDrive/Dissertation/GPT_data/Snorkel_data/400/data-00000-of-00001.arrow')
# Concatenate labels in 11 and 12 in the training dataset to the snorkel dataset
snorkel_ds_200 = concatenate_datasets([snorkel_ds_200,train_ds])
snorkel_ds_300 = concatenate_datasets([snorkel_ds_300,train_ds])
snorkel_ds_400 = concatenate_datasets([snorkel_ds_400,train_ds])
# Merge into one dataset
ds_snorkel = {'100': snorkel_ds_200, '200': snorkel_ds_300, '300': snorkel_ds_400}

In [ ]:
# Let's evaluate the improvement in model performance after data augementation
#Model
model_bert_100 = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                                    num_labels=len(label),id2label=id_to_label,
                                                                    label2id = label_to_id)
tokenizer_bert = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


References: Adapted from Antoniadis (2022)

In [ ]:
# Precision accuracy
import evaluate
f1_metric = evaluate.load("f1")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return f1_metric.compute(predictions=predictions, references=labels, average="weighted")

class LogAccumulatorCallback(TrainerCallback):
    def __init__(self):
        self.acc_logs = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero and ('loss' in logs or 'eval_loss' in logs):
            self.acc_logs.append(logs.copy())

def train_and_evaluate(train_ds, test_ds, output_dir):
    def tokenize(dataset):
        return tokenizer_bert(dataset['text'], truncation=True)

    train_ds = train_ds.map(tokenize, batched=True)
    test_ds = test_ds.map(tokenize, batched=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer_bert) # Dynamic padding

    model = model_bert_100
    logger = LogAccumulatorCallback()
    training_args = TrainingArguments(
        output_dir = output_dir,
        save_strategy="epoch",
        evaluation_strategy="epoch",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        weight_decay=0.01,
        learning_rate=1e-4,
        num_train_epochs=6,
        load_best_model_at_end=True,
        logging_steps=20,
        report_to='neptune'
    )

    trainer = Trainer(
        model=model, args=training_args,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        data_collator=data_collator,
        tokenizer=tokenizer_bert,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    model.save_pretrained(output_dir)
    metrics = trainer.evaluate()
    return metrics

# Import Neptune package

In [ ]:
! pip install neptune

In [ ]:
import neptune
from getpass import getpass
project='dducl/Dissertation-GPT'
os.environ["NEPTUNE_API_TOKEN"] = getpass("Enter your Neptune API token: ")
os.environ["NEPTUNE_PROJECT"] = project  # replace with your own

# Modeling

In [ ]:
# Model performance comnparison
performance = []
snorkel_dataset_num = ['100','200','300']
for i in range(0,len(snorkel_dataset_num)):
  num = snorkel_dataset_num[i]
  metrics = train_and_evaluate(ds_snorkel[num], test_ds, "augmented_snorkel"+ num)
  performance.append({
    "id": "augmented_snorkel" + num,
    "metrics": metrics,
    })
  i+=1

Map:   0%|          | 0/379 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,3.200800,2.688379,0.314886
2,2.468900,1.956657,0.441013
3,1.673100,1.461704,0.753283
4,1.164900,1.141291,0.812087
5,0.595100,0.974184,0.826840
6,0.458800,0.929307,0.826840


Map:   0%|          | 0/444 [00:00<?, ? examples/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.437700,0.774142,0.846032
2,0.221400,0.709257,0.815729
3,0.051400,0.620434,0.830880
4,0.034300,0.625904,0.830880
5,0.017200,0.603185,0.846032
6,0.018200,0.609847,0.846032


Map:   0%|          | 0/491 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.074900,0.767193,0.780303
2,0.020100,0.716628,0.801587
3,0.004100,0.760650,0.841162
4,0.002100,0.740321,0.841162
5,0.001800,0.742800,0.841162
6,0.001600,0.746188,0.841162


In [ ]:
baseline=[]
metrics = train_and_evaluate(train_ds, test_ds,'baseline')
baseline.append({
    "id": "baseline",
    "metrics": metrics
})

Map:   0%|          | 0/261 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,No log,1.088157,0.718398
2,0.671800,0.934900,0.710750
3,0.271100,0.902290,0.723775
4,0.086900,0.713030,0.802924
5,0.030100,0.706744,0.799461
6,0.021100,0.707528,0.798451


In [ ]:
label_category = list(id_to_label.keys())
label = list(id_to_label.values())
data = {'Label_id':label_category, 'Label':label}
labels = pd.DataFrame(data)

# Visualisation of distribution

In [ ]:
# Load data
snorkel_ds_200 = Dataset.from_file('/content/drive/MyDrive/Dissertation/GPT_data/Snorkel_data/200/data-00000-of-00001.arrow')
snorkel_ds_300 = Dataset.from_file('/content/drive/MyDrive/Dissertation/GPT_data/Snorkel_data/300/data-00000-of-00001.arrow')
snorkel_ds_400 = Dataset.from_file('/content/drive/MyDrive/Dissertation/GPT_data/Snorkel_data/400/data-00000-of-00001.arrow')
pd_sk_200 = snorkel_ds_200.to_pandas()
pd_sk_300 = snorkel_ds_300.to_pandas()
pd_sk_400 = snorkel_ds_400.to_pandas()
train_ds = train_ds.to_pandas()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
# Return the actual labels instead of label category
def return_to_label(list_cat):
  label = []
  for i in list_cat:
    label.append(id_to_label[i])
  return label

In [ ]:
# Snorkel
# Visualise frequency of labelling
# Initialise the subplot function using number of rows and columns
figure, axis = plt.subplots(2, 2,figsize=(10,15))
figure.tight_layout(pad=5.0,h_pad=5.0)
# plot NEO-graphs
sk_200 = pd_sk_200['labels'].value_counts(ascending=True).sort_values()
sk_300 = pd_sk_300['labels'].value_counts(ascending=True).sort_values()
sk_400 = pd_sk_400['labels'].value_counts(ascending=True).sort_values()
axis[0, 0].bar(return_to_label(list(sk_200.index)),list(sk_200.values))
axis[0, 1].bar(return_to_label(list(sk_300.index)),list(sk_300.values))
axis[1, 0].bar(return_to_label(list(sk_400.index)),list(sk_400.values))
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.3,
                    hspace=0.4)
figure.suptitle('Snorkel generated text distribution')
plt.draw()
# Define titles
axis[0,0].set_title('Synthetic 200 Samples')
axis[0,1].set_title('Synthetic 300 Samples')
axis[1,0].set_title('Synthetic 400 Samples')
# Define y_label
axis[0,0].set_ylabel('Number of samples')
axis[0,1].set_ylabel('Number of samples')
axis[1,0].set_ylabel('Number of samples')
# Format xtick
axis[0, 0].set_xticklabels(axis[0,0].get_xticklabels(), rotation=60, ha='right',fontsize=8)
axis[0, 1].set_xticklabels(axis[0,1].get_xticklabels(), rotation=60, ha='right',fontsize=8)
axis[1, 0].set_xticklabels(axis[1,0].get_xticklabels(), rotation=60, ha='right',fontsize=8)
axis[0,0].yaxis.set_major_formatter(FormatStrFormatter('%d'))
# Save figure
plt.savefig('snorkel_plot.png', dpi=300, bbox_inches='tight')